# Laboratorio 2

Integrantes del grupo:
1. Emmanuel Blanco - 202312743
2. Juan David Guzmán - 202320890

In [8]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, RobustScaler, PolynomialFeatures, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, validation_curve
from sklearn.utils import resample
import matplotlib.pyplot as plt


## Construcción del modelo de regresión polinomial

### Preparación de datos

Al igual que en el laboratorio anterior, el primer paso que realizamos en la preparación de datos es eliminar las filas de Id repetidos, y eliminar las filas con CVD Risk Score nulo.

In [2]:
training_data = pd.read_csv("./data/Datos_Lab_1.csv")
data = training_data.drop_duplicates(subset='Patient ID', keep='last')
print(f"Después de quitar duplicados: {data.shape[0]}")

data = data.dropna(subset=['CVD Risk Score'])
print(f"Después de quitar nulos en objetivo: {data.shape[0]}")

Después de quitar duplicados: 1376
Después de quitar nulos en objetivo: 1348


Una vez hechos estos primeros cambios hacemos la división de los datos para entrenamiento y prueba

In [3]:
target = 'CVD Risk Score'
x = data.drop(columns=[target])
y = data[target]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
print("Dimension datos entrenamiento:")
print(f"Training x: {x_train.shape}")
print(f"Training y: {y_train.shape}")
print("\nDimensión datos de prueba: ")
print(f"Test x: {x_test.shape}")
print(f"Test y: {y_test.shape}")

Dimension datos entrenamiento:
Training x: (1011, 23)
Training y: (1011,)

Dimensión datos de prueba: 
Test x: (337, 23)
Test y: (337,)


A partir de este punto construiremos el pipeline encargado de las demás transformaciones de los datos

In [13]:
# Se guardan en una lista las columnas que no consideraremos para este modelo
cols_to_drop = ['Patient ID', 'Date of Service', 'Blood Pressure (mmHg)','Blood Pressure Category', 'Height (cm)']

#Identificamos las columnas numéricas y categóricas
numeric_features = ['Age', 'Weight (kg)', 'Height (m)', 'BMI', 'Abdominal Circumference (cm)', 'Total Cholesterol (mg/dL)', 
                    'HDL (mg/dL)', 'Fasting Blood Sugar (mg/dL)','Waist-to-Height Ratio', 'Systolic BP', 'Diastolic BP', 
                    'Estimated LDL (mg/dL)']

categorical_features = ['Sex', 'Smoking Status', 'Diabetes Status', 'Physical Activity Level', 'Family History of CVD']

# Definimos la función que será usada para quitar las columnas que no necesitamos
def drop_columns(df):
    return df.drop(columns=cols_to_drop, errors='ignore')

dropper = FunctionTransformer(drop_columns)

# Definimos los transformadores para los dos tipos de datos
numeric_transformer = Pipeline(steps=[
    ("imputer",SimpleImputer(strategy="mean")),
    ("scaler",RobustScaler()),
    ("polynomial",PolynomialFeatures(degree=2)),
])

categorical_transformer = Pipeline(steps=[
    ("imputer",SimpleImputer(strategy="most_frequent")),
    ("onehot",OneHotEncoder(handle_unknown="ignore", drop="if_binary")),
])

# Unimos los transformadores de datos con un ColumnTransformer

preprocessor_polynomial = ColumnTransformer(transformers=[
    ("num",numeric_transformer,numeric_features),
    ("cat",categorical_transformer,categorical_features),
])

# definimos un función que se deshaga de los valores negativos
def corregir_negativos(df):
    df = df.copy()
    for column in numeric_features:
        df[column] = df[column].abs()
    return df

correcion_negs = FunctionTransformer(corregir_negativos)

# Finalmente montamos el pipeline para la regresión polinomial
pipeline_reg_polinom = Pipeline(steps=[
    ("dropper",dropper),
    ("correccion_negativos",correcion_negs),
    ("preprocesamiento",preprocessor_polynomial),
    ("modelo",LinearRegression())
])


Con el pipeline definido, ahora usaremos GridSearchCV para encontrar los mejores hiperparámetros para el modelo

In [20]:
#Definimos los parámetros que queremmos que el GridSearchCV pruebe

param_grid = {
    "preprocesamiento__num__scaler":[StandardScaler(),RobustScaler(),MinMaxScaler()],
    "preprocesamiento__num__imputer__strategy":["mean","median","most_frequent"],
    "preprocesamiento__num__polynomial__degree":[2,3,4]
}

grid_search = GridSearchCV(pipeline_reg_polinom,param_grid=param_grid,cv=10,scoring="neg_root_mean_squared_error",n_jobs=-1)
grid_search.fit(x_train, y_train)
grid_search.best_params_

print("Mejores parámetros obtenidos por GridSearch:\n")
print("Mejor estrategia de imputación para datos numéricos: ",grid_search.best_params_["preprocesamiento__num__imputer__strategy"])
print("Mejor grado para PolynomialFeautures: ",grid_search.best_params_["preprocesamiento__num__polynomial__degree"])
print("Mejor escalador de datos numéricos: ",grid_search.best_params_["preprocesamiento__num__scaler"])

mejor_modelo = grid_search.best_estimator_.named_steps["modelo"]
print("Número de coeficientes del modelo: ",len(mejor_modelo.coef_))

Mejores parámetros obtenidos por GridSearch:

Mejor estrategia de imputación para datos numéricos:  most_frequent
Mejor grado para PolynomialFeautures:  2
Mejor escalador de datos numéricos:  StandardScaler()
Número de coeficientes del modelo:  98


Ahora hacemos las predicciones y calculamos la métricas necesarias para validar la regresión polinomial

In [21]:
y_train_pred = grid_search.best_estimator_.predict(x_train)

#Calculamos las métricas
print(f'------ Modelo de regresión polinomial grado {grid_search.best_params_["preprocesamiento__num__polynomial__degree"]} ----')
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_train_pred)):.2f}")
print(f"MAE: {mean_absolute_error(y_train, y_train_pred):.2f}")
print(f'R²: {r2_score(y_train, y_train_pred):.2f}')

------ Modelo de regresión polinomial grado 2 ----
RMSE: 9.80
MAE: 3.75
R²: 0.17


## Generación de curvas de validación

Buscamos mostrar cómo cambia el error cuando se aumenta la complejidad del modelo (el grado del polinomio).Se entrena con ese proposito el mismo pipeline variando el grado de 1 a 4 con validación cruzada de 5 folds. 

In [ ]:

# Pipeline limpio para curvas de validación
preprocessor_curvas = ColumnTransformer(transformers=[
    ("num", Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("scaler", StandardScaler()),
        ("polynomial", PolynomialFeatures(include_bias=False)),
    ]), numeric_features),
    ("cat", Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", drop="if_binary")),
    ]), categorical_features),
])

pipeline_val_curve = Pipeline(steps=[
    ("dropper", dropper),
    ("correccion_negativos", correcion_negs),
    ("preprocesamiento", preprocessor_curvas),
    ("modelo", LinearRegression())
])

# Curva de validación: grado del polinomio
param_range_grado = [1, 2, 3, 4]

train_scores_vc, val_scores_vc = validation_curve(
    pipeline_val_curve,
    x_train, y_train,
    param_name="preprocesamiento__num__polynomial__degree",
    param_range=param_range_grado,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

train_mean_vc = -train_scores_vc.mean(axis=1)
train_std_vc  =  train_scores_vc.std(axis=1)
val_mean_vc   = -val_scores_vc.mean(axis=1)
val_std_vc    =  val_scores_vc.std(axis=1)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(param_range_grado, train_mean_vc, 'o-', color='steelblue', label='Error de entrenamiento (RMSE)')
ax.fill_between(param_range_grado,
                train_mean_vc - train_std_vc,
                train_mean_vc + train_std_vc,
                alpha=0.15, color='steelblue')
ax.plot(param_range_grado, val_mean_vc, 'o-', color='tomato', label='Error de validación (RMSE)')
ax.fill_between(param_range_grado,
                val_mean_vc - val_std_vc,
                val_mean_vc + val_std_vc,
                alpha=0.15, color='tomato')
ax.set_xlabel('Grado del polinomio', fontsize=12)
ax.set_ylabel('RMSE', fontsize=12)
ax.set_title('Curva de validación: Grado del polinomio vs RMSE', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

print(f"\nResumen curvas de validación:")
print(f"{'Grado':<10} {'RMSE Train':<18} {'RMSE Val':<18}")
print("-" * 46)
for g, tr, va in zip(param_range_grado, train_mean_vc, val_mean_vc):
    print(f"{g:<10} {tr:<18.4f} {va:<18.4f}")


Se produjo un grafico con dos lineas. Una roja que muestra el error de validación (baja, pero luego sube = sobreajuste) y una azul que señala el error de entrenamiento (siempre baja al subir el grado). Y pues la brecha entre las dos curvas te dice en qué grado empieza el sobreajuste.

## Construcción de modelos de regresión lineal regularizados

Pues son 2 modelos de reg. Lineal que tienen regularización (L2 y L1)

1. L2 o Ridge:

In [ ]:
preprocessor_lineal_ridge = ColumnTransformer(transformers=[
    ("num", Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
    ]), numeric_features),
    ("cat", Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", drop="if_binary")),
    ]), categorical_features),
])

pipeline_ridge = Pipeline(steps=[
    ("dropper", dropper),
    ("correccion_negativos", correcion_negs),
    ("preprocesamiento", preprocessor_lineal_ridge),
    ("modelo", Ridge())
])

param_grid_ridge = {
    "preprocesamiento__num__imputer__strategy": ["mean", "median", "most_frequent"],
    "preprocesamiento__num__scaler": [StandardScaler(), RobustScaler(), MinMaxScaler()],
    "modelo__alpha": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
}

grid_search_ridge = GridSearchCV(
    pipeline_ridge, param_grid=param_grid_ridge, cv=10,
    scoring="neg_root_mean_squared_error", n_jobs=-1
)
grid_search_ridge.fit(x_train, y_train)

print("=== Mejores parámetros Ridge ===")
print(f"  Estrategia imputación : {grid_search_ridge.best_params_['preprocesamiento__num__imputer__strategy']}")
print(f"  Escalador             : {grid_search_ridge.best_params_['preprocesamiento__num__scaler']}")
print(f"  Alpha                 : {grid_search_ridge.best_params_['modelo__alpha']}")

y_pred_ridge_train = grid_search_ridge.best_estimator_.predict(x_train)
print(f"\n--- Métricas Ridge (entrenamiento) ---")
print(f"RMSE : {np.sqrt(mean_squared_error(y_train, y_pred_ridge_train)):.4f}")
print(f"MAE  : {mean_absolute_error(y_train, y_pred_ridge_train):.4f}")
print(f"R²   : {r2_score(y_train, y_pred_ridge_train):.4f}")

Aquí penalizamos coeficientes grandes pero no los eliminamos. GridSearchCV en este caso busca el mejor alpha entre 7 valores, 3 escaladores y 3 estrategias de imputación.

2. L1 o Lasso

In [ ]:
preprocessor_lineal_lasso = ColumnTransformer(transformers=[
    ("num", Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
    ]), numeric_features),
    ("cat", Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", drop="if_binary")),
    ]), categorical_features),
])

pipeline_lasso = Pipeline(steps=[
    ("dropper", dropper),
    ("correccion_negativos", correcion_negs),
    ("preprocesamiento", preprocessor_lineal_lasso),
    ("modelo", Lasso(max_iter=10000))
])

param_grid_lasso = {
    "preprocesamiento__num__imputer__strategy": ["mean", "median", "most_frequent"],
    "preprocesamiento__num__scaler": [StandardScaler(), RobustScaler(), MinMaxScaler()],
    "modelo__alpha": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
}

grid_search_lasso = GridSearchCV(
    pipeline_lasso, param_grid=param_grid_lasso, cv=10,
    scoring="neg_root_mean_squared_error", n_jobs=-1
)
grid_search_lasso.fit(x_train, y_train)

print("\n=== Mejores parámetros Lasso ===")
print(f"  Estrategia imputación : {grid_search_lasso.best_params_['preprocesamiento__num__imputer__strategy']}")
print(f"  Escalador             : {grid_search_lasso.best_params_['preprocesamiento__num__scaler']}")
print(f"  Alpha                 : {grid_search_lasso.best_params_['modelo__alpha']}")

y_pred_lasso_train = grid_search_lasso.best_estimator_.predict(x_train)
print(f"\n--- Métricas Lasso (entrenamiento) ---")
print(f"RMSE : {np.sqrt(mean_squared_error(y_train, y_pred_lasso_train)):.4f}")
print(f"MAE  : {mean_absolute_error(y_train, y_pred_lasso_train):.4f}")
print(f"R²   : {r2_score(y_train, y_pred_lasso_train):.4f}")

# Variables seleccionadas por Lasso (coeficientes distintos de cero)
modelo_lasso_fitted = grid_search_lasso.best_estimator_.named_steps["modelo"]
preprocessor_lasso_fitted = grid_search_lasso.best_estimator_.named_steps["preprocesamiento"]

feature_names_cat = (preprocessor_lasso_fitted
                     .transformers_[1][1]
                     .named_steps["onehot"]
                     .get_feature_names_out(categorical_features)
                     .tolist())
all_feature_names = numeric_features + feature_names_cat
coef_lasso_df = pd.DataFrame({"Feature": all_feature_names,
                               "Coeficiente": modelo_lasso_fitted.coef_})
coef_nonzero = (coef_lasso_df[coef_lasso_df["Coeficiente"] != 0]
                .sort_values("Coeficiente", ascending=False))
print(f"\nVariables seleccionadas por Lasso (coef ≠ 0): "
      f"{len(coef_nonzero)} de {len(all_feature_names)}")
print(coef_nonzero.to_string(index=False))

# Gráfico de coeficientes Lasso
if len(coef_nonzero) > 0:
    fig, ax = plt.subplots(figsize=(10, max(4, len(coef_nonzero) * 0.4)))
    colors = ['steelblue' if c > 0 else 'tomato' for c in coef_nonzero["Coeficiente"]]
    ax.barh(coef_nonzero["Feature"], coef_nonzero["Coeficiente"], color=colors, alpha=0.8)
    ax.axvline(0, color='black', linewidth=0.8)
    ax.set_title("Coeficientes del modelo Lasso (variables seleccionadas)", fontsize=13)
    ax.set_xlabel("Coeficiente")
    ax.grid(axis='x', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()


Hacemos lo mismo que L2 pero además fOrzaMOS algunos coeficientes a exactamente cero, es decir, L1 selecciona variables automáticamente. Al final esto imprime qué variables quedaron y muestra un gráfico de barras con sus coeficientes.

## Construcción de un modelo de regresión polinomial regularizado

## Comparación y selección del mejor modelo

## Construcción de intervalos de confianza

## Análisis de resultados

### Análisis cuantitativo.

- ¿Cuál modelo obtuvo el mejor desempeño en el conjunto de test?

- ¿Coincide el mejor desempeño en test con el mejor promedio en validación cruzada? Si no coincide, ¿cuál puede ser la explicación?

- ¿El modelo con mejor métrica promedio es necesariamente el más adecuado? Justifica considerando también la desviación estándar del desempeño.

- Con base en las curvas de validación, ¿cómo cambia el error a medida que aumenta la complejidad? ¿En qué punto se evidencia sobreajuste?

- ¿Cómo afecta la regularización la magnitud y estabilidad de los coeficientes?

- ¿Los intervalos de confianza obtenidos mediante bootstrapping sugieren estabilidad o alta variabilidad en el desempeño? ¿Qué implicaciones tiene esto?

### Análisis cualitativo.

- ¿Qué variables fueron seleccionadas como más relevantes por el modelo Lasso?

- ¿Qué interpretación práctica tienen los coeficientes del modelo final en el contexto del riesgo cardiovascular?

- ¿Existen diferencias relevantes entre el modelo más preciso y el más interpretable?

- ¿Qué decisiones estratégicas podría tomar AlpesHearth a partir de los resultados obtenidos?

- ¿Mayor precisión implica necesariamente mayor valor para la organización?

- ¿Un modelo más complejo necesariamente genera mayor valor empresarial? Discute considerando interpretabilidad, estabilidad y costo de implementación.

### Reflexión conceptual.

- ¿Qué relación observas entre complejidad del modelo, capacidad de generalización y estabilidad del desempeño?

- ¿Qué fuentes de sesgo podrían estar presentes en los datos o en el proceso de modelado?

- Si el tamaño de muestra fuera mayor, ¿esperarías cambios en la estabilidad de los modelos? Explique.